In [ ]:
#installing packages
install.packages('themis')
install.packages('randomForest')
install.packages('readxl')
install.packages('rpart')
install.packages('vip')
install.packages('parsnip')
install.packages('rsample')
install.packages('workflows')
install.packages("lifecycle")
install.packages('yardstick')
install.packages("C50", repos="http://R-Forge.R-project.org") 
install.packages("ranger")
install.packages('yardstick')
install.packages('tune')
install.packages("tibble")
install.packages("plotly")
install.packages("C5.0")
install.packages("lightgbm")

In [ ]:
#loading dataset
credit=read.csv('credit.csv')

In [ ]:
library(plotly)
plot_ly(credit, x = ~Status, type = 'histogram', name = 'Status', marker = list(color = c("rgba(102, 194, 165,1)",
                                    "rgba(141, 160, 203,1)")))

In [ ]:
plot(credit$Amount, credit$Assets, xlab = "Amount", ylab = "Assets",pch = 19, col = factor(credit$Status))
legend("topright",
       legend = levels(factor(credit$Status)),
       pch = 19,
       col = factor(levels(factor(credit$Status))))

In [ ]:
library(recipes)
credit <- credit %>% 
  recipe(Status ~ ., data=.) %>% 
  # the next step only works with factor variables
  step_string2factor(Status) %>% 
  themis::step_upsample(Status) %>% 
  step_factor2string(Status) %>% 
  # keep characters
  prep(strings_as_factors=FALSE) %>% 
  juice()

In [ ]:
plot_ly(credit, x = ~Status, type = 'histogram', name = 'Status', marker = list(color = c("rgba(102, 194, 165,1)",
                                    "rgba(141, 160, 203,1)")))

In [ ]:
plot(credit$Amount, credit$Assets, xlab = "Amount", ylab = "Assets",pch = 19, col = factor(credit$Status))
legend("topright",
       legend = levels(factor(credit$Status)),
       pch = 19,
       col = factor(levels(factor(credit$Status))))

## Single Trees 

   \begin{align}
            \mathbf{\hat{p}}_mk = \frac{1}{N_m} \sum_{x_i \in R_m} I(y_i = k)
       \end{align}
    
where Nm is the number of items in region m and the summation counts the number of observations of class k in region m.

\begin{align}
\mathbf{\hat{f}(x)} = \sum_{m=1}^M \hat{c}_m I(x \in R_m)
\end{align}

where

\begin{align}
\mathbf{\hat{c}_m} = avg(y_i\mid x_i \in R_m)
\end{align}

is the average y value for the region.

### rpart 


. Regression, classification, Poisson, survival

. Formula interface only

    -Categorical inputs as character or factor
    -Categorical outcome as character or factor

. Handles missing data

In [ ]:
library(rpart)
mod_rpart <- rpart(Status ~ ., data=credit)

In [ ]:
library(rpart.plot)
rpart.plot(mod_rpart)

In [ ]:
library(vip)
vip(mod_rpart)

. minsplit: Minimum number of observations in a node for a split to be attempted

. minbucket: Minimum number of observations in a terminal node

. cp: A split must improve the fit by cp

. maxdepth: The maximum number of splits for terminal nodes

## Random Forest

\begin{align}
    \mathbf{\hat{f}} = \frac{1}{B} \sum_{b=1}^B \hat{f}^\mbox{*b}(x)
\end{align}

where $\hat{f}^\mbox{*b}(x)$ is a decision tree fit on the bth bootstrapped sample of data, with columns randomly selected for evaluation at each split.

### {random forest}

. Regression or classification

. formula interface

    -Categorical input data must be factors
    -Outcome must be a factor for classification

. x/y interface

    -x must be a dense matrix
    -y must be a factor for classification

. Does not handle missing data

In [ ]:
credit_imputed <- recipe(Status ~ ., data=credit) %>% 
  step_impute_knn(all_predictors()) %>% 
  prep(strings_as_factors=TRUE) %>% 
  juice()

In [ ]:
library(randomForest)
mod_randomForest <- randomForest(Status ~ ., data=credit_imputed)

In [ ]:
plot(mod_randomForest)

In [ ]:
vip(mod_randomForest)


. ntree: Number of trees to grow

. mtry: Number of candidate variables to check at each split

. replace: Whether to sample with replacement

. sampsize: Size of samples to draw

. nodesize: Maximum size of terminal nodes

. maxnodes: Maximum number of terminal nodes for each tree (complexity)

### {ranger} 

. Regression, classification, survival

. formula interface

    -Categorical input variables can be character or factor
    -Output must be a factor for classification

. x/y interface
    
    -Dense or sparse x
    -y argument must be factor for classification

. data.frame interface with name of output
. Does not handle missing values

In [ ]:
library(ranger)
mod_ranger <- ranger(Status ~ ., data=credit_imputed, importance='impurity')

In [ ]:
# no plotting method, causes error
plot(mod_ranger)

In [ ]:
vip(mod_ranger)

. num.trees: Number of trees to grow

. mtry: Number of candidate variables to check at each split

. max.depth: Maxim depth of any tree

. replace: Whether to sample with replacement

. sample.fraction: Size of samples to draw

. regularization.factor: Amount of penalization on gain

. regularization.usedepth: Whether to consider depth in penalization

. splitrule: Type of splitting to perform

## Boosted Trees 

\begin{align}
    \mathbf{\hat{y}_i}^t = \sum_{k=1}^t f_k(x_i) = \hat{y}_i^(t-1) + f_t(x_i)
\end{align}

where $f_k(x)$ is a tree and $f_k(x)$ is trained on the residuals of $f_{k-1} (x)$

### {gbm}

. Regression, classification, Poisson, survival, ranking
. formula interface (x/y interface in other function)

    -Categorical inputs must be factor
    -Binary outcome must be 0/1

. Handles missing data

In [ ]:
credit_gbm <- recipe(Status ~ ., data=credit) %>% 
  step_integer(Status, zero_based=TRUE) %>% 
  prep(strings_as_factors=TRUE) %>% 
  juice()

In [ ]:
library(gbm)
mod_gbm <- gbm(Status ~ ., data=credit_gbm, distribution='bernoulli', n.trees=100)

In [ ]:
# no plotting method, causes error
plot(mod_gbm, i.var=c('Seniority', 'Records', 'Income'))

In [ ]:
vip(mod_gbm)

. n.trees: Number of boosting rounds

. interaction.depth: Maximum depth of a tree in terms of the number of interactions

. n.minobsinnode: Minimum number of observations in terminal nodes

. shrinkage: Learning rate

. bag.fraction: Percentage of data to sample at each round of boosting

### {C5.0}

. Classification only
. formula interface

    -Categorical inputs as character or factor
    -Outcome of formula must be a factor

. x/y interface

    -Only dense matrices for x argument
    -y argument must be a factor

. Handles missing data

. Rule-based models

In [ ]:
library("C50")

In [ ]:
mod_C5.0_boost <- C5.0(factor(Status) ~ ., data=credit, trials=100)

In [ ]:
# no plotting method, causes error
plot(mod_C5.0_boost, subtree=3)

In [ ]:
vip(mod_C5.0_boost)

. minCases: Smallest number of samples that must be put in at least two of the splits

. trials: Number of boosting rounds

In [ ]:
mod_ranger <- ranger(Status ~ ., data=credit_imputed, importance='impurity')

In [ ]:
# no plotting method, causes error
plot(mod_ranger)

In [ ]:
vip(mod_ranger)

- num.trees: Number of trees to grow 
- mtry: Number of candidate variables to check at each split
- max.depth: Maxim depth of any tree
- replace: Whether to sample with replacement
- sample.fraction: Size of samples to draw
- regularization.factor: Amount of penalization on gain
- regularization.usedepth: Whether to consider depth in penalization
- splitrule: Type of splitting to perform

### Boosted Trees


\begin{align}
        \hat{y}^t_{i} =  \sum_{k=1}^t f_{k}(x_{i}) = \hat{y}_{i}^{t-1}+f_{t}(x_{i})  
\end{align}
  where $f_{k}(x)$  is a tree and $f_{k}(x)$ is trained on the residuals of $f_{k−1}(x)$





### {gbm}
- Regression, classification, Poisson, survival, ranking
- formula interface (x/y interface in other function)
        - Categorical inputs must be factor
        - Binary outcome must be 0/1
- Handles missing data

In [ ]:
credit_gbm <- recipe(Status ~ ., data=credit) %>% 
  step_integer(Status, zero_based=TRUE) %>% 
  prep(strings_as_factors=TRUE) %>% 
  juice()

In [ ]:
library(gbm)
mod_gbm <- gbm(Status ~ ., data=credit_gbm, distribution='bernoulli', n.trees=100)

In [ ]:
# no plotting method, causes error
plot(mod_gbm, i.var=c('Seniority', 'Records', 'Income'))

In [ ]:
vip(mod_gbm)

- n.trees: Number of boosting rounds
- interaction.depth: Maximum depth of a tree in terms of the number of interactions
- n.minobsinnode: Minimum number of observations in terminal nodes
- shrinkage: Learning rate
- bag.fraction: Percentage of data to sample at each round of boosting

### {C5.0}
- Classification only
- formula interface
        - Categorical inputs as character or factor
        - Outcome of formula must be a factor
- x/y interface
        - Only dense matrices for x argument
        - y argument must be a factor
- Handles missing data
- Rule-based models

In [ ]:
mod_C5.0_boost <- C5.0(factor(Status) ~ ., data=credit, trials=100)

In [ ]:
# no plotting method, causes error
plot(mod_C5.0_boost, subtree=3)

In [ ]:
vip(mod_C5.0_boost)

- minCases: Smallest number of samples that must be put in at least two of the splits
- trials: Number of boosting rounds

### {xgboost}
- Regression, classification, ranking
- Only x/y interface via xgb.DMatrix()
        - x can be dense or sparse matrix, or file(s) on disc
        - Outcome must be 0/1 for binary classification
- Handles missing values
- Handles imbalanced data
- GLM models: penalized
- CPU or GPU

In [ ]:
rec_xg <- recipe(Status ~ ., data=credit) %>% 
  step_integer(Status, zero_based=TRUE) %>% 
  step_dummy(all_nominal(), one_hot=TRUE) %>% 
  prep()

x_xg <- juice(rec_xg, all_predictors(), composition='dgCMatrix')
y_xg <- juice(rec_xg, all_outcomes(), composition='matrix')

library(xgboost)
credit_xg <- xgb.DMatrix(data=x_xg, label=y_xg)

In [ ]:
mod_xgboost <- xgb.train(
  data=credit_xg, 
  objective='binary:logistic',
  nrounds=100, 
  watchlist=list(train=credit_xg), 
  print_every_n=10
)

In [ ]:
dygraphs::dygraph(mod_xgboost$evaluation_log)

In [ ]:
xgb.plot.multi.trees(mod_xgboost)

In [ ]:
vip(mod_xgboost)

- nrounds: Number of boosting rounds
- eta: Learning rate
- gamma: Minimum loss reduction required to make a split
- max_depth: The most splits for any one tree
- min_child_weight: Minimum weight needed to make a split (larger: more conservative)
- subsample: Percent of rows to sample for each tree
- colsample_bytree: Percent of columns to randomly sample for each tree
- num_parallel_tree: How many trees to grow in each round
        - Allows for boosted pseudo random forests
- early_stopping_rounds: Number of rounds without improvement before stopping - early_stopping_rounds

### {lightgbm}
- Regression, classification
- Only x/y interface via lgb.Dataset()
        - x can be dense or sparse matrix, or file(s) on disc
        - Categorical inputs can be dummies or numeric encoding
        - Outcome must be 0/1 for binary classification
        - Outcome can be dense or sparse
- Handles missing values
- Handles imbalanced data
- CPU or GPU

In [ ]:
rec_lgb <- recipe(Status ~ ., data=credit) %>% 
  step_integer(all_nominal(), zero_based=TRUE) %>%
  prep()

credit_char <- credit %>% select(-Status) %>% 
    purrr::map_lgl(~is.character(.x)) %>% which()

x_lgb <- juice(rec_lgb, all_predictors(), composition='dgCMatrix')
y_lgb <- juice(rec_lgb, all_outcomes(), composition='matrix')

library(lightgbm)
credit_lgb <- lgb.Dataset(data=x_lgb, label=y_lgb, categorical_feature=credit_char)

In [ ]:
mod_lightgbm <- lightgbm(data=credit_lgb, nrounds=100, obj='binary', 
                         eval_freq=10, is_unbalance=TRUE)

In [ ]:
# no plotting method, causes error
plot(mod_lightgbm)

In [ ]:
mod_lightgbm %>% lgb.importance() %>% lgb.plot.importance()

- num_iterations: Number of boosting rounds
- learning_rate: Learning rate
- max_depth: The most splits for any one tree
- bagging_fraction: Percent of rows to sample for each tree
- feature_fraction: Percent of columns to randomly sample for each tree
- boosting: Type of boosting to perform

### Other Models
- RuleFit
- Catboost
- BART
- TensorFlow


### So Many Different Interfaces
- formula
- matrices
        - Dense
        - Sparse
- data.frame plus vector
- Proprietary formats (xgb.DMatrix, lgb.Dataset)
- characters vs factors vs dummy variables vs numeric encoding

- {rpart}: formula
- {C5.0}: formula or dense matrices
- {randomForest}: formula or dense matrix input with factor outcome
- {ranger}: formula or dense/sparse matrix input with factor outcome
- {gbm}: formula or dense/sparse matrix input with numeric outcome using gbm.fit()
- {xgboost}: dense/sparse matrix input with numeric outcome inside xgb.DMatrix object
- {lightgbm}: dense/sparse matrix input with numeric outcome inside lgb.Dataset object.size

- {rpart}: Categorical variables as factor or character
- {C5.0}: Categorical variables as factor or character or dummy variables
- {randomForest}: Categorical variables as factor or dummy variables
- {ranger}: Categorical variables as factor or character or dummy variables
- {gbm}: Categorical variables as factor or dummy variables
- {xgboost}: Categorical variables as dummy variables
- {lightgbm}: Categorical variables as dummy variables or numeric encoding

## {tidymodels}

In [ ]:
library(parsnip)

In [ ]:
spec_rpart <- decision_tree(mode='classification') %>% 
  set_engine('rpart')

spec_rpart

In [ ]:
spec_C50 <- decision_tree(mode='classification') %>% 
  set_engine('C5.0')

spec_C50

In [ ]:
spec_randomForest <- rand_forest(mode='classification') %>% 
  set_engine('randomForest')

spec_randomForest

In [ ]:
spec_ranger <- rand_forest(mode='classification') %>% 
  set_engine('ranger')

spec_ranger

In [ ]:
spec_C50_boost <- boost_tree(mode='classification') %>% 
  set_engine('C5.0')

spec_C50_boost

In [ ]:
spec_xgboost <- boost_tree(mode='classification') %>% 
  set_engine('xgboost')

spec_xgboost

## Feature Engineering with {recipes}

In [ ]:
library(rsample)
set.seed(28676)
data_split <- initial_split(credit, prop=.9, strata='Status')
train <- training(data_split)
test <- testing(data_split)

In [ ]:
rec_rpart <- recipe(Status ~ ., data=train) %>% 
  themis::step_upsample(Status) %>% 
  step_other(all_nominal(), -Status, other='misc')

In [ ]:
rec_C50 <- recipe(Status ~ ., data=train) %>% 
  themis::step_upsample(Status) %>% 
  step_other(all_nominal(), -Status, other='misc')

In [ ]:
rec_randomForest <- recipe(Status ~ ., data=train) %>% 
  themis::step_upsample(Status) %>% 
  step_string2factor(all_nominal(), -Status) %>% 
  step_knnimpute(all_predictors()) %>% 
  step_other(all_nominal(), -Status, other='misc')

In [ ]:
rec_ranger <- recipe(Status ~ ., data=train) %>% 
  themis::step_upsample(Status) %>% 
  step_string2factor(all_nominal(), -Status) %>% 
  step_knnimpute(all_predictors()) %>% 
  step_other(all_nominal(), -Status, other='misc')

In [ ]:
rec_C50_boost <- recipe(Status ~ ., data=train) %>% 
  themis::step_upsample(Status) %>% 
  step_other(all_nominal(), -Status, other='misc')

In [ ]:
rec_xgboost <- recipe(Status ~ ., data=train) %>% 
  themis::step_upsample(Status) %>%
  step_other(all_nominal(), -Status, other='misc') %>% 
  step_dummy(all_nominal(), -Status, one_hot=TRUE)

## {workflows} 

In [ ]:
library(workflows)

In [ ]:
work_rpart <- workflow() %>% 
  add_recipe(rec_rpart) %>% 
  add_model(spec_rpart)

work_C50 <- workflow() %>% 
  add_recipe(rec_C50) %>% 
  add_model(spec_C50)

In [ ]:
work_randomForest <- workflow() %>% 
  add_recipe(rec_randomForest) %>% 
  add_model(spec_randomForest)

work_ranger <- workflow() %>% 
  add_recipe(rec_ranger) %>% 
  add_model(spec_ranger)

In [ ]:
work_C50_boost <- workflow() %>% 
  add_recipe(rec_C50_boost) %>% 
  add_model(spec_C50_boost)

work_xgboost <- workflow() %>% 
  add_recipe(rec_xgboost) %>% 
  add_model(spec_xgboost)

## Fit the Models

In [ ]:
# trees
fit_rpart <- work_rpart %>% fit(data=train)
fit_C50 <- work_C50 %>% fit(data=train)

# random forests
fit_randomForest <- work_randomForest %>% fit(data=train)
fit_ranger <- work_ranger %>% fit(data=train)

# boosted trees
fit_C50_boost <- work_C50_boost %>% fit(data=train)
fit_xgboost <- work_xgboost %>% fit(data=train)

## How did we do?
• Accuracy
• Speed

In [ ]:
models <- list(
  'rpart'=work_rpart
  , 'C50'=work_C50
  , 'randomForest'=work_randomForest
  , 'ranger'=work_ranger
  , 'C50_boost'=work_C50_boost
  , 'xgboost'=work_xgboost
)

In [ ]:
quality_metric <- yardstick::metric_set(yardstick::roc_auc)

In [ ]:
quality <- models %>% 
  purrr::map(
    ~tune::last_fit(.x, split=data_split, metrics=quality_metric)
  )

In [ ]:
model_assesments <- quality %>% 
  purrr::map_dfr(tune::collect_metrics) %>% 
  mutate(Model=names(models)) %>% 
  select(Model, AUC=.estimate) %>% 
  bind_cols(quality %>% bind_rows())

In [ ]:
library(plotly)
plot_ly(
  data=model_assesments %>% arrange(AUC) %>% mutate(Model=factor(Model, levels=Model)), 
  x=~Model, y=~AUC) %>% 
  add_lines(marker=list(color=~AUC)) %>% add_annotations(text=~Model)

In [ ]:
library(bench)

# run the speed test
model_times <- press(
  model=models,
  mark(fit(model[[1]], data=train), iterations=5)
)

# combine with AUC
model_checks <- model_times %>% 
  select(Time=median, Memory=mem_alloc) %>% 
  bind_cols(model_assesments %>% select(Model, AUC))

In [ ]:
plot_ly(
  data=model_checks %>% arrange(Time) %>% mutate(Model=factor(Model, levels=Model)), 
  x=~Model, y=~Time) %>% 
  add_lines(marker=list(color=~Time)) %>% add_annotations(text=~Model)

In [ ]:
model_checks %>% 
  plot_ly(x=~Time, y=~AUC) %>% 
  add_lines(marker=list(color=~AUC)) %>% 
  add_annotations(text=~Model) %>% layout(showlegend = FALSE)

## Takeaways

• So many interfaces to know

• Nuances for each model

• {xgboost} is nearly the fastest

• {xgboost} is nearly the most correct

• For this data

• On my computer

• Everything is easier with {tidymodels}

## Thank you!